use simplified data 2

data source: ./simplified_data/simplified_data2.csv

data 處理:

data v2 刪掉所有文字input

deleted 回診,西藥

## train 方法:

1個1個藥train,

每個model 只train 一種藥

只train 出現次數多於250 的藥 (=n剛好剩10種藥)

加weight 到under representative class

by

class_weights = compute_class_weight('balanced', np.unique(y_train), y_train)

class_weight_dict = dict(enumerate(class_weights))


In [26]:
import numpy as np
import pandas as pd
import random
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from sklearn.utils.class_weight import compute_sample_weight

# custom import 
from utility_file import my_utilities as myutil

In [27]:
def ReadData(FILENAME):
    data = pd.read_csv(FILENAME, encoding='ANSI')
    print("in ReadData")
    print("type of data:", type(data))
    print(f'Shape of data = ({data.shape[0]} rows, {data.shape[1]} cols).')
    return data

def SplitXY(data, data_left_bound, label_left_bound, data_right_bound=None, label_right_bound=None):
    """label_left_bound: 第一個藥材的col no."""
    # Body status: 1~3, Diagnosis: 4~7, Symptom: 11~124
    # Prescription: 125~226
    # split_X = list(range(0, label_left_bound))
    # split_Y = list(range(label_left_bound, len(data.columns)))
    if label_right_bound is None:
        label_right_bound = len(data.columns)
    if data_right_bound is None:
        data_right_bound = label_left_bound
    X = data.iloc[1:, list(range(data_left_bound, data_right_bound))]
    y = data.iloc[1:, list(range(label_left_bound, label_right_bound))]
    
    # Debug
    print("SplitXY:")
    print(f'Shape of X = ({X.shape[0]} rows, {X.shape[1]} cols). First 10 data of X:')
    print(X.iloc[:10, :10])
    print(f'Shape of y = ({y.shape[0]} rows, {y.shape[1]} cols). First 10 data of y:')
    print(y.iloc[:10, :10])
    return X, y

def SplitNparr(original_arr: np.ndarray, train_portion: float)->tuple:
    data_len =len(original_arr)
    train_len = int(data_len * train_portion)
    indices = list(range(data_len))
    random.shuffle(indices)
    train_idx = indices[:train_len]
    validate_idx = indices[train_len:]
    training_arr  = original_arr[train_idx]
    validation_arr = original_arr[validate_idx]

    return (training_arr,validation_arr)


def Split1Df(data: pd.DataFrame,  train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - data: any data
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - data_train: data for the training set.
    - data_val: data for the validation set.

    """
    if random_state is not None:
        random.seed(random_state)
    data_len = len(data)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]


    data_train, data_val = data.loc[train_idx], data.loc[val_idx]
   
    return data_train, data_val

def SplitBothXy_Df(X: pd.DataFrame, y: pd.DataFrame, train_size: float, random_state: int=None):
    """
    Split the data into training and validation sets.

    Param:
    - X: Features 
    - y: Target variable
    - train_size: Proportion of in train set.
    - random_state: Seed 

    return:
    - X_train: Features for the training set.
    - X_val: Features for the validation set.
    - y_train: Target variable for the training set.
    - y_val: Target variable for the validation set.
    """
    if random_state is None:
        random_state = 42
    random.seed(random_state)
    data_len = len(X)
    val_len = int(data_len * (1-train_size))
    indices = list(range(data_len))
    random.shuffle(indices)
    val_idx = indices[:val_len]
    train_idx = indices[val_len:]
    print("type of val_idx", type(val_idx))
    print("type of train_idx", type(train_idx))
    print("len of val_idx", len(val_idx))

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    assert len(X_train) == len(y_train)
    assert len(X_val) == len(y_val)
    assert(X_train.shape[1] == X_val.shape[1])
    assert(y_train.shape[1] == y_val.shape[1])
    return X_train, X_val, y_train, y_val



In [28]:
def DeleteMedicine(y, threshold: int=250):
    """
        delete 出現次數< threshold 的藥
    """
    for col in y.columns:
        # print(f'Number of {col} is {y[col].sum()}.')
        if y[col].sum() < threshold:
            y = y.drop(col, axis=1)
    # Debug
    print(f'DeleteMedicine: shape of y is {y.shape}.')
    return y

In [29]:
data_pd = ReadData("./simplified_data/simplified_data2.csv")

in ReadData
type of data: <class 'pandas.core.frame.DataFrame'>
Shape of data = (797 rows, 215 cols).


In [30]:
first_medicine_idx = None
#first_medicine_idx = 113
first_medicane = '麻黃'

for i in range(len(data_pd.columns.tolist())):
    if data_pd.columns.tolist()[i] == first_medicane:
        first_medicine_idx = i
        break
print("first_medicine: ",data_pd.columns[first_medicine_idx])
print("first_medicine_idx: ", first_medicine_idx)
print("should be 麻黃")

# split data into X and y
# x= all symptoms, diagnosis, body status
# y= all medince
X,y = SplitXY(data_pd, data_left_bound=2, data_right_bound=first_medicine_idx, label_left_bound=first_medicine_idx)

# drop 出現次數少於threshold 的藥in y
y = DeleteMedicine(y, threshold=250)

train_X, val_X, train_y, val_y = SplitBothXy_Df(X, y, 0.8)

X_np = train_X.values.astype('float64')
X_val_np = val_X.values.astype('float64')


num_col_x = X_np.shape[1]


first_medicine:  麻黃
first_medicine_idx:  113
should be 麻黃
SplitXY:
Shape of X = (796 rows, 111 cols). First 10 data of X:
    肺癌  胰臟癌  肝癌  腺癌  攝護腺癌  骨癌  淋巴癌  胃癌  腦瘤  肝炎
1    0    0   0   0     0   0    0   0   0   0
2    0    0   0   0     0   0    0   0   0   0
3    0    0   0   0     0   0    0   0   0   0
4    0    0   0   0     0   0    0   0   0   0
5    0    0   0   0     0   0    0   0   0   0
6    0    0   0   0     0   0    0   0   0   0
7    0    0   0   0     0   0    0   0   0   0
8    0    0   0   0     0   0    0   0   0   0
9    0    0   0   0     0   0    0   0   0   0
10   0    0   0   0     0   0    0   0   0   0
Shape of y = (796 rows, 102 cols). First 10 data of y:
    麻黃  桂枝  荊芥  防風  細辛  白芷  生薑  辛夷  葛根  升麻
1    0   1   0   0   1   1   0   0   0   0
2    0   1   0   0   0   0   1   0   0   0
3    0   0   0   0   0   0   0   0   0   0
4    0   1   0   0   0   0   1   1   0   0
5    0   1   0   0   0   0   0   0   0   0
6    0   0   0   0   0   0   0   0   0   0
7    

In [31]:
# debug message 
print("train_X.shape: ", train_X.shape)
print("train_y.shape: ", train_y.shape)

print("X transformed to np array")
print("type of X_np:", type(X_np))
print("shape of X_np:", X_np.shape)
print("shape of train y:", train_y.shape)
print("type of X_val_np:", type(X_val_np))
print("shape of X_val_np:", X_val_np.shape)

print("number of col in (train) x:",num_col_x)
print("number of medicine in y: ", train_y.shape[1])



train_X.shape:  (637, 111)
train_y.shape:  (637, 10)
X transformed to np array
type of X_np: <class 'numpy.ndarray'>
shape of X_np: (637, 111)
shape of train y: (637, 10)
type of X_val_np: <class 'numpy.ndarray'>
shape of X_val_np: (159, 111)
number of col in (train) x: 111
number of medicine in y:  10


## compute class weight

In [32]:

class_weights = compute_sample_weight(class_weight='balanced', y=train_y)
class_weight_dict = dict(enumerate(class_weights))


In [33]:
# build and compile model

model = Sequential([
    Dense (units=16, input_shape=(num_col_x,), activation='relu'),
    Dense (units=32, activation='relu'), 
    Dense (units=32, activation='relu'), 
    Dense (units=2, activation='softmax')
])
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 16)                1792      
                                                                 
 dense_5 (Dense)             (None, 32)                544       
                                                                 
 dense_6 (Dense)             (None, 32)                1056      
                                                                 
 dense_7 (Dense)             (None, 2)                 66        
                                                                 
Total params: 3,458
Trainable params: 3,458
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer=Adam(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [35]:
all_res=[]
result_df_dict={}   # dict of df
#loss_acc_dict={}    # dict of loss and accuracy of each medicine
accuracy_dict={}    # dict of accuracy of each medicine

for i in range(train_y.shape[1]):
#for i in range(2):
    res = {}
    res_list =[]
    chosen_col = train_y.iloc[:,i].copy()
    assert(isinstance(chosen_col, pd.Series))
    assert(len(chosen_col) == len(train_y))
    print("processing medicine:", chosen_col.name)
    chosen_y_np = chosen_col.values.astype('float64')

    y_val_chosen_col = val_y.iloc[:,i].copy()
    assert(isinstance(y_val_chosen_col, pd.Series))
    assert(len(y_val_chosen_col) == len(val_y))
    #y_val_chosen_col_np = y_val_chosen_col.values.astype('float64')

    # fit model for this medicine
    model.fit(x=X_np, y=chosen_y_np,
        class_weight=class_weight_dict,  # <---- add class weight
          batch_size=50, epochs=100, shuffle=True, verbose=0)
    
    # make prediction for this medicine
    predictions = model.predict(X_val_np)
    print(type(predictions))
    print("predictions.shape:", predictions.shape)
    
    df_predictions = pd.DataFrame(predictions, columns=["predicted as 0", "predicted as 1"])
    df_predictions["predicted value"] = np.where(df_predictions["predicted as 0"] > df_predictions["predicted as 1"], 0, 1)
    df_predictions["ground truth"] = y_val_chosen_col.values
     # create new column that checks if "predicted value" is equal to "ground truth"
    df_predictions["is_correct"] = df_predictions["predicted value"] == df_predictions["ground truth"]

    accuracy = df_predictions["is_correct"].mean()
    accuracy_dict[chosen_col.name] = accuracy  
    result_df_dict[chosen_col.name] = df_predictions
    

print("training done.")




processing medicine: 桂枝


5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 柴胡
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 黃芩
5/5 [==============================] - 0s 5ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 茯苓
5/5 [==============================] - 0s 3ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 澤瀉
5/5 [==============================] - 0s 2ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 附子
5/5 [==============================] - 0s 2ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 甘草
5/5 [==============================] - 0s 4ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processing medicine: 當歸
5/5 [==============================] - 0s 2ms/step
<class 'numpy.ndarray'>
predictions.shape: (159, 2)
processi

## Handle result


In [36]:

folder_name = "train_1_med_result"
predicted_value_index = None
ground_truth_index = None

# calculate f1 score of for each medicine

for key, df in result_df_dict.items():
    # create 'TP/FP/TN/TN/FN' column

    if predicted_value_index is None:
        predicted_value_index = df.columns.get_loc('predicted value')
    if ground_truth_index is None:
        ground_truth_index = df.columns.get_loc('ground truth')


    df['TP/FP/TN/TN/FN'] = df.apply(lambda row: 'TP' if row[predicted_value_index] == 1 and row[ground_truth_index] == 1 else 
                                 ('FP' if row[predicted_value_index] == 1 and row[ground_truth_index] == 0 else 
                                  ('FN' if row[predicted_value_index] == 0 and row[ground_truth_index] == 1 else 'TN')), axis=1)

# count occurrences of 'TP', 'FP', 'FN', and 'TN'
counts = df['TP/FP/TN/TN/FN'].value_counts()
# get number of 'TP', 'FP', 'FN', and 'TN'
num_tp = counts.get('TP', 0)
num_fp = counts.get('FP', 0)
num_fn = counts.get('FN', 0)
num_tn = counts.get('TN', 0)

f1_score = 2 * num_tp / (2 * num_tp + num_fp + num_fn)
    
for key, df in result_df_dict.items():
    print(f"DataFrame for {key}:")
    myutil.print_df(df)
    myutil.df_to_csv(df, save_path="top10med_result", file_prefix=key)
   




C:\Users\taliah\AppData\Local\Temp\ipykernel_8824\400165267.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df['TP/FP/TN/TN/FN'] = df.apply(lambda row: 'TP' if row[predicted_value_index] == 1 and row[ground_truth_index] == 1 else
C:\Users\taliah\AppData\Local\Temp\ipykernel_8824\400165267.py:18: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ('FP' if row[predicted_value_index] == 1 and row[ground_truth_index] == 0 else
C:\Users\taliah\AppData\Local\Temp\ipykernel_8824\400165267.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treat

DataFrame for 桂枝:

+-----+------------------+------------------+-------------------+----------------+--------------+------------------+
|     |   predicted as 0 |   predicted as 1 |   predicted value |   ground truth | is_correct   | TP/FP/TN/TN/FN   |
|-----+------------------+------------------+-------------------+----------------+--------------+------------------|
|   0 |                0 |                1 |                 1 |              0 | False        | FP               |
|   1 |                0 |                1 |                 1 |              0 | False        | FP               |
|   2 |                0 |                1 |                 1 |              0 | False        | FP               |
|   3 |                0 |                1 |                 1 |              0 | False        | FP               |
|   4 |                0 |                1 |                 1 |              1 | True         | TP               |
|   5 |                1 |                0 |

In [37]:
print("total medical case: " , len(val_y))
total_med_case =  len(val_y)
assert((num_tp+num_fp+num_fn+num_tn)== len(val_y))
print("f1 score:", f1_score)
print(f"TP: {num_tp}, percentage = {num_tp/total_med_case}")
print(f"FP: {num_fp}, percentage = {num_fp/total_med_case}")
print(f"FN: {num_fn}, percentage = {num_fn/total_med_case}")
print(f"TN: {num_tn}, percentage = {num_tn/total_med_case}")
print(accuracy_dict)

total medical case:  159
f1 score: 0.7467811158798283
TP: 87, percentage= 0.5471698113207547
FP: 40, percentage= 0.25157232704402516
FN: 19, percentage= 0.11949685534591195
TN: 13, percentage= 0.08176100628930817
{'桂枝': 0.44025157232704404, '柴胡': 0.6415094339622641, '黃芩': 0.6163522012578616, '茯苓': 0.6352201257861635, '澤瀉': 0.5345911949685535, '附子': 0.4968553459119497, '甘草': 0.6163522012578616, '當歸': 0.5031446540880503, '白芍': 0.6037735849056604, '炙甘草': 0.6289308176100629}


In [38]:


import statistics

mean_accuracy = statistics.mean(accuracy_dict.values())
print("mean accuracy: ", mean_accuracy)



mean accuracy:  0.5716981132075472


In [39]:
# from utility_file import my_utilities as myutil
# import json
# myutil.dict_to_txt(loss_acc_dict, save_path=f"./result/{folder_name}/")
# with open('./result/res.txt', 'w', encoding='utf-8') as file:
#     json.dump(loss_acc_dict, file, indent=2)